In [1]:
# The following is needed to import Nanoscopy until it is released as a full package.
import sys
sys.path.append('..')

import nanoscopy as nano

# # Load some commonly-used packages
# import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt
# import pandas as pd
# import math

# Introduction
The goal of this tutorial is to teach users how to import and perform basic levelling of SPM images using Nanoscopy.

The first step to working with an SPM image is to import it. Nanoscopy currently relies on other packages to load the SPM image data from files. When the data has been loaded from a file, Nanoscopy stores the data for future use in a special data container, called "SPMImage", which is defined in the Nanoscopy package itself. This data container stores the different data channels contained in the SPM image file, as well as important metadata from the file.

# Importing data
SPM data importing in Nanoscopy makes use of the "io" module in the "spm" subpackage. This module contains a convenience function which will attempt to load the data from its file based on the file extension as long as the extension is one that is recognized by the importing algorithm. The function is called "read". The read function can take either the path of a file to be loaded or the path to a directory of files to be loaded. Regardless, the read function always returns a list of SPMImage items, with one item per file loaded.

Let's load an example data file, which happens to be an STM image produced by Nanonis control hardware (which uses the SXM file extension):

In [5]:
# Specify the file to be imported
filepath_sxm = '../ExampleDataFiles/STM_WTip_WSe2_SL333-Set2-Crystal2_031.sxm'

Image = nano.spm.io.read(filepath_sxm)[0]




# def batch_HHCF(path):
#     spm_images = nano.spm.io.read(path)
#     HHCF_data = []
#     for image in spm_images:
#         for index in [0,1]:
#             levelled_image = nano.spm.process.terrace_level(image.data['Z'][index])
#             HHCF_data.append(nano.spm.analysis.calculate_HHCF(levelled_image, image.params['width']))
#     return spm_images, HHCF_data

Processing ../ExampleDataFiles/STM_WTip_WSe2_SL333-Set2-Crystal2_031.sxm...DONE


In the code above, we included a "[0]" after the read function to select the first (and only) item in the list returned by read. Thus, "Image" is an SPMImage class object rather than a list of SPMImage objects.

# Accessing Data Channels
The SPMImage class has several properties. The first property is a dictionary containing the image data channels.

In [10]:
print(Image.)

{'bias': 1.0, 'setpoint_value': 3e-11, 'setpoint_unit': 'A', 'width': 1e-07, 'height': 1e-07, 'channels': ['Current', '(A);Z', '(m);LIX', '1', 'omega', '(A)'], 'scan_direction': 'down', 'date_time': datetime.datetime(2021, 8, 23, 18, 56, 56)}
